In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import pytz

# Merge all daily PV measurements into 1 file

In [ ]:
for root, dirs, files in os.walk('./solar_data/pv_day'):
    day_nr = 0
    first_df = True
    df_total =pd.DataFrame()
    for filename in files:
        # extract date from filename
        d = filename[-14:-4]
        
        # load file in pandaframe
        f = os.path.join(root,filename)
        pv_day = pd.read_csv(f,';',thousands='.')

        # Cleaning -> change: col names + remove NaN
        # we have to re-assign because original df is not changed with reneme
        pv_day = pv_day.rename(columns={' ':'Time','Tribout Peter Brugge / Power / Mean Values  [kW]0':'avg kW'})
        # we replace empty cells with 0 (when no sun PV stores NaN instead of 0) 
        pv_day['avg kW'].fillna(0,inplace=True)
        pv_day['avg kW'] = pv_day['avg kW'].astype(int)

        # add col with timestamp + convert to string
        pv_day['Date'] = d
        pv_day['Time'] = pv_day['Time'] + ':00'
        pv_day['Date'] = pv_day['Date'].astype(str)
        pv_day['Time'] = pv_day['Time'].astype(str)
        
        pv_day['timestamp'] = pd.to_datetime(pv_day['Date'] + 'T' + pv_day['Time'], format='%Y-%m-%dT%H:%M:%S')
        infer_dst = np.array([False] * pv_day.shape[0])  
        pv_day['timestamp'] = pv_day['timestamp'].dt.tz_localize(tz='Europe/Brussels',ambiguous=infer_dst, nonexistent='shift_forward')
        #pv_day['dt_iso'] = pv_day['timestamp']
        pv_day.set_index('timestamp', inplace = True)
        
        
        df_total = df_total.append(pv_day)
# sort index so we write first timestamp first            
df_total = df_total.sort_index()


/Users/tribp/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# Save to file
df_total.to_csv('pv_day.csv')

In [ ]:
df_total.head()

,Time,avg kW,Date
timestamp,,,
2020-01-01 00:00:00+01:00,00:00:00,0,2020-01-01
2020-01-01 00:15:00+01:00,00:15:00,0,2020-01-01
2020-01-01 00:30:00+01:00,00:30:00,0,2020-01-01
2020-01-01 00:45:00+01:00,00:45:00,0,2020-01-01
2020-01-01 01:00:00+01:00,01:00:00,0,2020-01-01


# Read weather file, filter used fields and merge some fields

In [ ]:
weather = pd.read_csv('./solar_data/weather/weather_2020.csv', ';')

/Users/tribp/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
weather = weather.drop(['timezone','dt_iso','city_name','lat','lon','feels_like','temp_min','temp_max','sea_level','grnd_level','rain_1h','rain_3h','snow_1h','snow_3h','weather_icon'], axis=1)


In [ ]:
weather['timestamp'] = pd.to_datetime(weather['dt'],unit='s')
weather = weather.drop(['dt'], axis=1)
infer_dst = np.array([False] * weather.shape[0])  
weather['timestamp'] = weather['timestamp'].dt.tz_localize(tz='UTC',ambiguous=infer_dst, nonexistent='shift_forward')
weather['timestamp'] = weather['timestamp'].dt.tz_convert(tz='Europe/Brussels')
weather.set_index('timestamp', inplace = True)

# reduce unnecessary pression 
weather['temp'] = weather['temp'].astype(int)
weather['pressure'] = weather['pressure'].astype(int)
weather['humidity'] = weather['humidity'].astype(int)
weather['wind_speed'] = weather['wind_speed'].astype(int)
weather['wind_deg'] = weather['wind_deg'].astype(int)
weather['clouds_all'] = weather['clouds_all'].astype(int)
weather['weather_id'] = weather['weather_id'].astype(int)

In [ ]:
# First we restrict number of atmospheric conditions in 'weather'
weather_type = {
    '211': 'thunderstorm',
    '500': 'light rain',
    '501': 'moderate rain',
    '502': 'heavy rain',
    '600': 'light snow',
    '601': 'snow',
    '701': 'fog',
    '800': 'clear sky',
    '801': 'few clouds',
    '802': 'scattered clouds',
    '803': 'broken clouds',
    '804': 'overcast clouds'
}
def map_weather_code(code):
    if code >=200 and code < 300:
        # Thunderstorm
        return 211
    elif code >=300 and code <= 500:
        #Drizzle -> light rain
        return 500
    elif code >=502 and code <=531:
        # Heavy Rain
        return 502
    elif code >=601 and code <=622:
        # Snow
        return 601
    elif code >=701 and code <=781:
        # Fog
        return 601
    else:
        return code
weather['weather_id']=weather['weather_id'].apply(lambda x:map_weather_code(x))
weather['weather']=weather['weather_id'].apply(lambda x:weather_type[str(x)])
weather = weather.drop(['weather_main','weather_description'], axis=1)

In [ ]:
weather.head()

,temp,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_id,weather
timestamp,,,,,,,,
2020-01-01 01:00:00+01:00,276,1033,97,5,120,81,803,broken clouds
2020-01-01 02:00:00+01:00,276,1033,92,5,130,20,601,snow
2020-01-01 03:00:00+01:00,276,1032,92,5,120,40,601,snow
2020-01-01 04:00:00+01:00,276,1031,91,4,140,20,601,snow
2020-01-01 05:00:00+01:00,276,1031,91,4,130,40,601,snow


# Combine PV measurents + weather

In [ ]:
# join both dataframes
all_data = df_total.join(weather)
# prepare, clean, fill
# we start on first record where we have weather AND PV data (00:00:00 1/1/2000 UTC)
all_data = all_data[4:]
# fill last record NaN with previous
all_data.loc['2020-12-31 23:45:00+01:00',['temp','pressure','humidity','wind_speed','wind_deg','clouds_all','weather_id','weather']]=all_data.loc['2020-12-31 23:00:00+01:00',['temp','pressure','humidity','wind_speed','wind_deg','clouds_all','weather_id','weather']].values


In [ ]:
all_data.head()

,Time,avg kW,Date,temp,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_id,weather
timestamp,,,,,,,,,,,
2020-01-01 01:00:00+01:00,01:00:00,0,2020-01-01,276.0,1033.0,97.0,5.0,120.0,81.0,803.0,broken clouds
2020-01-01 01:15:00+01:00,01:15:00,0,2020-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 01:30:00+01:00,01:30:00,0,2020-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 01:45:00+01:00,01:45:00,0,2020-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 02:00:00+01:00,02:00:00,0,2020-01-01,276.0,1033.0,92.0,5.0,130.0,20.0,601.0,snow


In [ ]:
# fill forward weather and weather_id for xx:15 / xx:30 / xx:45 since we only have hourly data
all_data['weather_id'] = all_data['weather_id'].interpolate(method='ffill')
all_data['weather'] = all_data['weather'].interpolate(method='ffill')

# interpolate all numerical values for these missing quarter values
all_data = all_data.interpolate(method='linear')


In [ ]:
all_data

,Time,avg kW,Date,temp,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_id,weather
timestamp,,,,,,,,,,,
2020-01-01 01:00:00+01:00,01:00:00,0,2020-01-01,276.0,1033.0,97.00,5.00,120.0,81.00,803.0,broken clouds
2020-01-01 01:15:00+01:00,01:15:00,0,2020-01-01,276.0,1033.0,95.75,5.00,122.5,65.75,803.0,broken clouds
2020-01-01 01:30:00+01:00,01:30:00,0,2020-01-01,276.0,1033.0,94.50,5.00,125.0,50.50,803.0,broken clouds
2020-01-01 01:45:00+01:00,01:45:00,0,2020-01-01,276.0,1033.0,93.25,5.00,127.5,35.25,803.0,broken clouds
2020-01-01 02:00:00+01:00,02:00:00,0,2020-01-01,276.0,1033.0,92.00,5.00,130.0,20.00,601.0,snow
...,...,...,...,...,...,...,...,...,...,...,...
2020-12-31 22:45:00+01:00,22:45:00,0,2020-12-31,275.0,1006.0,83.75,1.75,205.0,78.75,804.0,overcast clouds
2020-12-31 23:00:00+01:00,23:00:00,0,2020-12-31,275.0,1006.0,84.00,2.00,210.0,75.00,803.0,broken clouds
2020-12-31 23:15:00+01:00,23:15:00,0,2020-12-31,275.0,1006.0,84.00,2.00,210.0,75.00,803.0,broken clouds


# Calculate and insert clear_sky_pv

In [ ]:
import matplotlib.pyplot as plt
import pvlib
from pvlib import clearsky, atmosphere, solarposition, irradiance
from pvlib.location import Location

In [ ]:
def get_irradiance(site_location, start_date, stop_date, tilt, surface_azimuth):
    # Creates one day's worth of 10 min intervals
    times = pd.date_range(start=start_date, end=stop_date, freq='15min', tz=site_location.tz)
    # Generate clearsky data using the Ineichen model, which is the default
    # The get_clearsky method returns a dataframe with values for GHI, DNI,
    # and DHI
    clearsky = site_location.get_clearsky(times)
    # Get solar azimuth and zenith to pass to the transposition function
    solar_position = site_location.get_solarposition(times=times)
    # Use the get_total_irradiance function to transpose the GHI to POA
    POA_irradiance = irradiance.get_total_irradiance(
        surface_tilt=tilt,
        surface_azimuth=surface_azimuth,
        dni=clearsky['dni'],
        ghi=clearsky['ghi'],
        dhi=clearsky['dhi'],
        solar_zenith=solar_position['apparent_zenith'],
        solar_azimuth=solar_position['azimuth'])
    # Return DataFrame with only GHI and POA
    return pd.DataFrame({'GHI': clearsky['ghi'],
                         'POA': POA_irradiance['poa_global']})

In [ ]:
# Info of solar installation
pv_tilt = 40
azimuth = 170

lat = 51.11
lon = 3.10
site = Location(51.11, 3.10, 'Europe/Brussels', 70, 'Brugge')
brugge = pytz.timezone('Europe/Brussels')

# Enter your total installed PV peak power, max power of invertor + max sun irradiance at your lat/lon
total_installed_PV_panels = 7480
max_PV_invertor = 5040
max_sun_irradiance_m2 =913

In [ ]:
# Calculate irradiance and apply correction + clip at max PV invertor value
#start_time = all_data.index[0].value
#stop_time = all_data.index[-1].value
start_time = '2020-01-01 01:00:00'
stop_time = '2020-12-31 23:45:00'

# we assume max sun power =+/- 1000 Watt/m2 and have a installation of peak 7480 Watt so we multiply by 7.48
cf = 0.97
P_peak = total_installed_PV_panels / max_sun_irradiance_m2
year_irradiance = get_irradiance(site, start_time,stop_time, pv_tilt, azimuth)
year_irradiance['clear_sky'] = cf * P_peak* year_irradiance['POA']

# we clip the produced power to the max of the inverter (5040 Watt in this case)
year_irradiance['clear_sky'] = year_irradiance['clear_sky'].apply(lambda x: max_PV_invertor if x > max_PV_invertor else x)


In [ ]:
year_irradiance.head()

,GHI,POA,clear_sky
2020-01-01 01:00:00+01:00,0.0,0.0,0.0
2020-01-01 01:15:00+01:00,0.0,0.0,0.0
2020-01-01 01:30:00+01:00,0.0,0.0,0.0
2020-01-01 01:45:00+01:00,0.0,0.0,0.0
2020-01-01 02:00:00+01:00,0.0,0.0,0.0


In [ ]:
# Clean dataframe ready to merge
year_irradiance = year_irradiance.drop(['GHI','POA'], axis=1)
# join both dataframes
final = all_data.join(year_irradiance)

In [ ]:
final

,Time,avg kW,Date,temp,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_id,weather,clear_sky
2020-01-01 01:00:00+01:00,01:00:00,0,2020-01-01,276.0,1033.0,97.00,5.00,120.0,81.00,803.0,broken clouds,0.0
2020-01-01 01:15:00+01:00,01:15:00,0,2020-01-01,276.0,1033.0,95.75,5.00,122.5,65.75,803.0,broken clouds,0.0
2020-01-01 01:30:00+01:00,01:30:00,0,2020-01-01,276.0,1033.0,94.50,5.00,125.0,50.50,803.0,broken clouds,0.0
2020-01-01 01:45:00+01:00,01:45:00,0,2020-01-01,276.0,1033.0,93.25,5.00,127.5,35.25,803.0,broken clouds,0.0
2020-01-01 02:00:00+01:00,02:00:00,0,2020-01-01,276.0,1033.0,92.00,5.00,130.0,20.00,601.0,snow,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-31 22:45:00+01:00,22:45:00,0,2020-12-31,275.0,1006.0,83.75,1.75,205.0,78.75,804.0,overcast clouds,0.0
2020-12-31 23:00:00+01:00,23:00:00,0,2020-12-31,275.0,1006.0,84.00,2.00,210.0,75.00,803.0,broken clouds,0.0
2020-12-31 23:15:00+01:00,23:15:00,0,2020-12-31,275.0,1006.0,84.00,2.00,210.0,75.00,803.0,broken clouds,0.0
2020-12-31 23:30:00+01:00,23:30:00,0,2020-12-31,275.0,1006.0,84.00,2.00,210.0,75.00,803.0,broken clouds,0.0


In [ ]:
# Save to file
final.to_csv('pv_dataset.csv')